In [27]:
# Import libraries
# t-test libraries at the end 

import requests
import pandas as pd
import json
import os
import arcpy
import numpy as np
from scipy import stats 
from numpy.random import seed 
from numpy.random import randn 
from numpy.random import normal 
from scipy.stats import ttest_1samp
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt

In [28]:
# define the workspace so the files write to this spot
workspace = r'Z:\Helpdesk\Shana\GIS 5572\5572Final\5572Final.gdb'

In [34]:
# Define field names to pull from the census data into the dataframe
field_names = ['County', 'P005_calc_pct0004', 'mean_delay_saturday', 'mean_saturday_status', 'Point_Count']

In [35]:
# Create a numpy array in order to create a dataframe
arrayr = arcpy.da.FeatureClassToNumPyArray('RamseyTract_SummarizeWithin', field_names)

In [36]:
# Create a numpy array in order to create a dataframe
arrayh = arcpy.da.FeatureClassToNumPyArray('HennepinTract_SummarizeWithin', field_names)

In [37]:
dfr = pd.DataFrame(arrayr)
dfh = pd.DataFrame(arrayh)



In [39]:
df_t = pd.concat([dfr, dfh], axis=0)

df_t.head()

,County,P005_calc_pct0004,mean_delay_saturday,mean_saturday_status,Point_Count
0,Ramsey County,5.1,2.635135,0.689189,74
1,Ramsey County,19.4,2.141935,0.774194,155
2,Ramsey County,16.1,1.778689,0.860656,122
3,Ramsey County,16.8,1.463768,0.811594,69
4,Ramsey County,34.7,0.863636,0.647727,88


In [40]:
df_t.reset_index(drop=True, inplace=True)

## Student TTest
In this anaylsis, my null hypothesis is that delays in geographies (block groups or tracts) with high percentage of Black population is the same as in block groups or tracts with lower percentanges of Black population.

alternative hypothesis: dealys in the geographies (block groups or tracts) with high percentage of Black population is HIGHER or LOWER than in block groups or tracts with lower percentanges of Black population.

Reference: https://thedatascientist.com/how-to-do-a-t-test-in-python/

In [41]:
columns = df_t.columns.tolist()
print(columns)

['County', 'P005_calc_pct0004', 'mean_delay_saturday', 'mean_saturday_status', 'Point_Count']


In [42]:
# Calculate the mean of the delay in the census tracts
mean_value = df_t['P005_calc_pct0004'].mean()
std_deviation = df_t['P005_calc_pct0004'].std()

print("Mean:", mean_value)
print("Standard Deviation:", std_deviation)

Mean: 13.614225053078556
Standard Deviation: 13.680549532646735


In [43]:
# create a new df with only the block groups with the highest percentage of African American population (the top 25% )
percent_pop = df_t['P005_calc_pct0004']

# Calculate the 75th percentile value
threshold = pd.Series(percent_pop).quantile(0.75)

# Select rows where the field value is greater than or equal to the threshold
top = df_t[percent_pop >= threshold]

In [44]:
# count the number of rows in the subset
num_rows = len(top)  # Using len() function
print("Number of rows:", num_rows)

Number of rows: 118


In [46]:
# count the number of rows in the whole population
num_rows = len(df_t)  # Using len() function
print("Number of rows:", num_rows)

Number of rows: 472


In [47]:
# Finding the mean of the sample selection - the  25% of Census Tracts with highest population of African Americans
mean_value_sample = top['P005_calc_pct0004'].mean()
std_deviation_sample = top['P005_calc_pct0004'].std()

print("Mean:", mean_value_sample)
print("Standard Deviation:", std_deviation_sample)

Mean: 32.644067796610166
Standard Deviation: 13.840974364032023


In [48]:
# Select only the census tracts that have stops in them. Eliminate all block groups that have 0 in the Point Count field
# Yes, I should've done this way at the beginning 
df_t = df_t[df_t['Point_Count'] != 0]
top = top[top['Point_Count'] != 0]

In [49]:
# Create lists from the "mean_delay_saturday" field to use in the ttest
subset = top['mean_delay_saturday'].tolist()
whole = df_t['mean_delay_saturday'].tolist()

In [50]:
# Calculate the ttest
t_statistic, p_value = ttest_ind(subset, whole)

print("t-statistic:", t_statistic)
print("p-value:", p_value)

t-statistic: -0.17741341727730953
p-value: 0.8592604274152805


## Results
With the results of:
* t-statistic: -0.17741341727730953
* p-value: 0.8592604274152805

We can conclude that the null hypothesis is correct: there is no statistical difference in the mean delay time between Census Tracts with high African American population and the set of block groups as a whole.. 